In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import torch
print(torch.cuda.is_available())  # Returns True if GPU is available
print(torch.cuda.current_device())  # Displays the current GPU device
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # Displays GPU name


True
0
NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import load_dataset

import torch
import transformers
import pandas as pd
from tqdm import tqdm

# Initialize tqdm for pandas
tqdm.pandas(disable=True)


In [4]:
# Load tokenizer and model from Hugging Face
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
# Load IMDb dataset
dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [8]:
# Print the first 5 samples from the IMDb training dataset
for i in range(5):
    print(dataset["train"][i])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [9]:
# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16= True
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle().select(range(1000)),
    eval_dataset=tokenized_datasets["test"].shuffle().select(range(500)),
)

In [11]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.289626
2,No log,0.248758
3,No log,0.276478


TrainOutput(global_step=189, training_loss=0.2922398804356812, metrics={'train_runtime': 773.4159, 'train_samples_per_second': 3.879, 'train_steps_per_second': 0.244, 'total_flos': 789333166080000.0, 'train_loss': 0.2922398804356812, 'epoch': 3.0})

In [13]:
# Save the Fine-Tuned Model
model.save_pretrained("fine_tuned_bert")
tokenizer.save_pretrained("fine_tuned_bert")

('fine_tuned_bert\\tokenizer_config.json',
 'fine_tuned_bert\\special_tokens_map.json',
 'fine_tuned_bert\\vocab.txt',
 'fine_tuned_bert\\added_tokens.json',
 'fine_tuned_bert\\tokenizer.json')

In [14]:
# Load the fine-tuned model
classifier = pipeline("text-classification", model="fine_tuned_bert", device=0 if torch.cuda.is_available() else -1)

Device set to use cuda:0


In [16]:
# Test with sample text
print(classifier("This movie is shit as good"))

[{'label': 'LABEL_0', 'score': 0.8246008157730103}]


In [19]:
# Define updated training arguments
training_args = TrainingArguments(
    output_dir="./results_main",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Updated batch size
    num_train_epochs=3,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle().select(range(1000)),
    eval_dataset=tokenized_datasets["test"].shuffle().select(range(500)),
)



In [20]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.282503
2,No log,0.425826
3,No log,0.362950


TrainOutput(global_step=375, training_loss=0.20224094645182292, metrics={'train_runtime': 516.1277, 'train_samples_per_second': 5.813, 'train_steps_per_second': 0.727, 'total_flos': 789333166080000.0, 'train_loss': 0.20224094645182292, 'epoch': 3.0})